In [1]:
import numpy as np
import pandas as pd
from math import sqrt
from random import sample
import copy

In [2]:
# Мутация
def mutation(population_number, n, k):
    index_list = []
    for i in range(population_number):
        chromosome = sample(range(n - 1), k)
        chromosome.sort()
        index_list.append(chromosome)
    return index_list


# Проверка листа на одинаковость
def check_equal(index_list):
    new_list = []
    for element in index_list:
        if element not in new_list:
            new_list.append(element)
    return len(new_list) == 1

# Проверка хромосомы на дубликаты
def check_duplicates(chromosome):
    new_list = []
    for element in chromosome:
        if element not in new_list:
            new_list.append(element)
    return len(new_list) != len(chromosome)


def crossover(population_number, father_list, mother_list, index_list, temp_list):
    for i in range(population_number):
        j = father_list[i]
        z = mother_list[i]
        for k in range(len(index_list[i])):
            if k % 2 == 0:
                index_list[i][k] = temp_list[j][k]
            else:
                index_list[i][k] = temp_list[z][k]
    return index_list


def get_mothers(population_number, father_list, prob_list):
    mother_list = []
    for i in range(population_number):
        m = tournament_selection(population_number, prob_list)
        while m[0] == father_list[i]:
            m = tournament_selection(population_number, prob_list)
        mother_list.append(m[0])
    return mother_list


# Турнирная селекция
def tournament_selection(population_number, prob_list):
    resList = []
    for i in range(population_number):
        someList = sample(range(len(prob_list)), population_number // 2)
        res = 0
        maxNumber = -1
        for i in range(len(someList)):
            if (prob_list[someList[i]] > maxNumber):
                res = someList[i]
                maxNumber = prob_list[someList[i]]
        resList.append(res)
    return resList


# Расчёт QCFE - Quality criterion of features ensemble (критерий качества ансамбля признаков)
def calculate_qcfe(chromosome, data, n, k):
    corr_list = []
    for i in chromosome:
        corr_list.append(data[i][n])
    first_value = k * np.mean(corr_list)  # степень зависимости признаков

    corr_list = []
    i = 0
    j = 1
    while i != len(chromosome) - 1:
        z = j
        while z < len(chromosome):
            corr_list.append(data[i][z])
            z += 1
        i += 1
        j += 1
    second_value = sqrt(k + k * (k - 1) * np.mean(corr_list))  # степень независимости признаков

    return first_value / second_value


# Генерирование первой популяции
def generate_first_population(population_number, n, k):
    index_list = []
    for i in range(population_number):
        chromosome = sample(range(n - 1), k)
        chromosome.sort()
        index_list.append(chromosome)
    return index_list


def genetic_algorithm(data, col_names, k, population_number):
    n = len(col_names)  # общее количество признаков

    # лист индексов, который будет первой популяцией наших особей
    index_list = generate_first_population(population_number, n, k)

    best_qcfe = -1
    ensemble = []

    # ГЕНЕТИЧЕСКИЙ АЛГОРИТМ
    for index in range(200):
        qcfe_list = []
        for chromosome in index_list:
            qcfe = calculate_qcfe(chromosome, data, n, k)
            qcfe_list.append(qcfe)

        max_qcfe = -1
        max_index = -1
        for i in range(len(index_list)):
            if qcfe_list[i] > max_qcfe:
                max_qcfe = qcfe_list[i]
                max_index = i
        if max_qcfe > best_qcfe:
            best_qcfe = max_qcfe
            ensemble = index_list[max_index]

        prob_list = []
        for qcfe in qcfe_list:
            prob_list.append(qcfe / sum(qcfe_list))

        # Селекция особей (отцов и матерей)
        father_list = tournament_selection(population_number, prob_list)
        mother_list = get_mothers(population_number, father_list, prob_list)

        # Кроссовер
        temp_list = copy.deepcopy(index_list)
        index_list = crossover(population_number, father_list, mother_list, index_list, temp_list)
        
        for chromosome in index_list:
            chromosome.sort()
        
        for m in range(len(index_list)):
            if check_duplicates(index_list[m]):
                index_list[m] = sample(range(n - 1), k)
                index_list[m].sort()
        
        if check_equal(index_list):
            index_list = mutation(population_number, n, k)

    return best_qcfe, ensemble



In [13]:
data = pd.read_excel('/home/mhoncharuk/Education/liver_disease_recognizer/notebooks/GLCM_Diff_corr.xlsx')  # загрузка данных
data = data.fillna(0)
population_number = 16  # количество особей в популяции
max_qcfe = 0
best_ensemble = []
# best_k = k
qcfe_list = []
col_names = [str(i) for i in range(data.shape[0] - 1)]
for k in range(5, 31):
    print(k)
    best_qcfe, ensemble = genetic_algorithm(data, col_names, k, population_number)
    if best_qcfe > max_qcfe:
        max_qcfe = best_qcfe
        best_ensemble = ensemble
        best_k = k
    qcfe_list.append(best_qcfe)

# вывод результатов
text = ''
for index in best_ensemble:
    text += col_names[index] + ';'
print('k = ', best_k)
print(text)
print(max_qcfe)
print()

5


/home/mhoncharuk/Education/liver_disease_recognizer/pvl/lib/python3.6/site-packages/ipykernel_launcher.py:125: RuntimeWarning: invalid value encountered in double_scalars


KeyboardInterrupt: 

In [14]:
data


,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,118,119,120,121,122,123,124,125,126,127
0,0,1.000000,0.014608,0.317716,0.393082,0.400441,0.284986,0.488219,0.332447,0.131735,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.014608,1.000000,0.040865,0.402592,0.165862,0.018948,0.148674,0.179364,0.020173,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.317716,0.040865,1.000000,0.381946,0.419686,0.461052,0.744687,0.197425,0.443451,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.393082,0.402592,0.381946,1.000000,0.231620,0.385407,0.433834,0.287023,0.284317,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.400441,0.165862,0.419686,0.231620,1.000000,0.367257,0.479504,0.590731,0.466764,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,123,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124,124,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,126,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
best_ensemble
check_duplicates(best_ensemble)

False

In [65]:
for qfce in qcfe_list:
    print(qfce)

0.29533344098645986
0.26653444642413504
0.3359776559752297
0.38348900582728335
0.33418899724956475
0.3779356346466372
0.4163265790284426
0.38029948309629186
0.40752568085402296
0.37639111969226796
0.3807609785759849
0.3951375864630551
0.29829568083775043
0.30631743121172583
0.31328948372315196
0.3174209099845269
0.3185877201428388
0.3130230842657023
0.3127687323987955
0.3245036347386351
0.3425077762091823
0.3832739846047545
0.3315959835052767
0.3562597176773215
0.33037916375269927
0.3259851072809612
